## Analysis: Price Behavior of Agricultural Commodities Amid Typhoons

Typhoons disrupt agricultural supply chains in the Philippines, damaging crops, limiting transport, and reducing market availability. Using AgriPrice data, we analyzed how commodity prices respond to these shocks using three indicators: **price spikes**, **volatility**, and **lag or delay in adjustment**.

In [89]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime
import warnings
import os

warnings.filterwarnings('ignore', category=UserWarning)

CLEANED_DATA_DIR = "data_cleaned"

TYPHOON_FILE = os.path.join(CLEANED_DATA_DIR, "Typhoon_Dataset - Sheet8.csv")

CHART_WIDTH = 1000
CHART_HEIGHT = 400
X_DOMAIN_START = '2021-01-01'
X_DOMAIN_END = '2025-09-01'

PRICE_FILES = [
    "Rootcrops-Food-Prices.csv",
    "Leafy-Vegetables-Food-Prices.csv",
    "Fruits-Food-Prices.csv",
    "Fruit-Vegetables-Food-Prices.csv",
    "Condiments-Food-Prices.csv"
]

In [90]:
try:
    df_typhoons_raw = pd.read_csv(TYPHOON_FILE)
    df_typhoons_raw['Date_Entered_PAR'] = pd.to_datetime(
        df_typhoons_raw['Date Entered PAR'], errors='coerce'
    )
    df_typhoons = df_typhoons_raw.dropna(subset=['Date_Entered_PAR'])
    df_typhoons = df_typhoons[['Typhoon Name', 'Date_Entered_PAR', 'Classification', 'Peak Intensity']]
    print("Typhoon data loaded and processed.")
except FileNotFoundError:
    print(f"Typhoon data not found at {TYPHOON_FILE}.")
    df_typhoons = pd.DataFrame()

Typhoon data loaded and processed.


In [91]:
def create_typhoon_chart(price_file_name, crop_type_name, top_n=5):
    if df_typhoons.empty:
        print("No typhoon data available.")
        return None

    file_path = os.path.join(CLEANED_DATA_DIR, price_file_name)
    try:
        df_prices = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Missing file: {file_path}")
        return None

    df_prices['Date'] = pd.to_datetime(
        df_prices['Year'].astype(str) + ' ' + df_prices['Month'], 
        format='%Y %B'
    )

    min_date, max_date = df_prices['Date'].min(), df_prices['Date'].max()
    df_typhoons_filtered = df_typhoons[
        (df_typhoons['Date_Entered_PAR'] >= min_date) & 
        (df_typhoons['Date_Entered_PAR'] <= max_date)
    ]

    commodity_volatility = df_prices.groupby('Commodity_Name')['Retail_Price'].std().reset_index()
    top_commodities = commodity_volatility.sort_values('Retail_Price', ascending=False).head(top_n)['Commodity_Name'].tolist()

    df_national_avg = df_prices[df_prices['Commodity_Name'].isin(top_commodities)].groupby(
        ['Date', 'Commodity_Name']
    ).agg(Retail_Price=('Retail_Price', 'mean')).reset_index()

    base = alt.Chart(df_national_avg).encode(
        x=alt.X('Date:T', title='Date'),
        y=alt.Y('Retail_Price:Q', title='National Avg Price (PHP/KG)'),
        color=alt.Color('Commodity_Name:N', title=f'{crop_type_name} Commodity')
    )

    lines = base.mark_line(point=True).encode(
        tooltip=[
            alt.Tooltip('Date:T', title='Month', format="%Y %B"),
            alt.Tooltip('Commodity_Name:N', title='Commodity'),
            alt.Tooltip('Retail_Price:Q', title='Avg Price (PHP)', format=',.2f')
        ]
    )

    typhoon_rules = alt.Chart(df_typhoons_filtered).mark_rule(
        color='red', strokeDash=[5, 3]
    ).encode(
        x='Date_Entered_PAR:T',
        tooltip=[
            alt.Tooltip('Date_Entered_PAR:T', title='Typhoon Entered PAR', format="%Y-%m-%d"),
            'Typhoon Name:N',
            'Classification:N',
            'Peak Intensity:N'
        ]
    )

    chart = (lines + typhoon_rules).properties(
        width=800,
        height=400,
        title=f'National Average {crop_type_name} Prices vs Typhoon Events (Top {top_n} Volatile Commodities)'
    ).interactive(bind_y=False)

    return chart

In [92]:
def create_and_display_chart_for_file(file_name):
    commodity_key = file_name.replace('-Food-Prices.csv', '')
    crop_type_name = commodity_key.replace('-', ' ').title().replace('Fruit Vegetables', 'Fruit and Vegetables')
    chart = create_typhoon_chart(file_name, crop_type_name)
    return chart

In [93]:
create_and_display_chart_for_file(PRICE_FILES[0])

alt.LayerChart(...)

### Analysis: National Average Rootcrop Prices vs Typhoon Events

- Prices of **carrot** and **potato** show noticeable spikes during or shortly after clusters of typhoon events, especially around the **July–October** period each year.  
- These spikes suggest that **typhoon events may disrupt production or supply chains**, leading to higher market prices.  
- In contrast, **gabi** and **singkamas** display relatively more stable price trends, indicating **lower sensitivity to typhoon disruptions** compared to other rootcrops.  
- There’s a recurring **seasonal pattern**, where prices peak in the second half of the year, which aligns with the country’s **typhoon season**.  
- The consistent timing of these spikes supports the idea that **climate shocks have short-term but noticeable impacts on food prices**.

In [95]:
create_and_display_chart_for_file(PRICE_FILES[1])

alt.LayerChart(...)

### Analysis: National Average Leafy Vegetables Prices vs Typhoon Events

- **Cabbage** exhibits the **most significant price fluctuations**, with sharp spikes often aligning with clusters of typhoon events (especially during July–October).  
- **Pechay native** and **alugbati** also shows noticeable variability, though not as extreme as cabbage.  
- **Kangkong**, and **malunggay** remain relatively more stable, with smaller price movements across the same period.  
- These seasonal spikes strongly correspond to the **peak typhoon season**, suggesting that extreme weather has a **direct impact on price volatility**, particularly for more **climate-sensitive crops** like cabbage.  
- Cabbage’s high volatility may be due to its **vulnerability to heavy rain and strong winds**, affecting both yield and post-harvest quality.  

In [97]:
create_and_display_chart_for_file(PRICE_FILES[2])

alt.LayerChart(...)

### Analysis: National Average Fruits Prices vs Typhoon Events

- **Grapes** consistently have the **highest price levels** among the top 5 fruits, but their price trend remains relatively stable over time despite typhoon clusters.  
- **Mango (Kalabao and Pico)** show **noticeable price fluctuations**, with spikes that often coincide with typhoon periods, particularly around **July–October**.  
- **Lanzones** displays a **notable deep dip in price** around **late 2024 to early 2025**, which coincides with **a cluster of typhoon events**.  
  - This suggests that **excess supply during peak harvest** combined with **possible distribution or quality issues due to typhoons** may have **driven prices downward** rather than up.  
  - Lanzones may be particularly **vulnerable to post-harvest spoilage** when transport and storage are disrupted.  
- **Avocado** prices remain fairly stable with minor fluctuations, indicating less sensitivity to typhoon shocks compared to other fruits.  
- Overall, fruit prices tend to **fluctuate less sharply than vegetables**, though some varieties (like mango) clearly respond to seasonal and climatic disruptions.

In [99]:
create_and_display_chart_for_file(PRICE_FILES[3])

alt.LayerChart(...)

### Analysis: National Average Fruit and Vegetables Prices vs Typhoon Events

- **Tomato** exhibits the **most pronounced price spikes**, especially around **late 2024 and mid-2025**, which align with multiple **typhoon clusters**. This indicates a **strong climate sensitivity**, likely due to the crop’s high perishability and vulnerability to heavy rain and wind.  
- **Eggplant** and **okra** display **moderate fluctuations**, with some upward price movements following typhoon periods but not as sharp as tomato.  
- **Ampalaya (bitter gourd)** shows **fluctuations over time**, but these are **less clearly tied to typhoon events**, suggesting that other factors (e.g., seasonal cycles or production conditions) may be influencing its price more than extreme weather.  
- **Cucumber (pipino)** remains relatively **stable**, showing the least volatility among the five commodities.  
- Overall, the data suggests that **highly perishable crops like tomato** are **more vulnerable to typhoon disruptions**, while crops like cucumber and ampalaya appear **less directly affected**.  

In [101]:
create_and_display_chart_for_file(PRICE_FILES[4])

alt.LayerChart(...)

### Analysis: National Average Condiments Prices vs Typhoon Events

- **Onion red creole and onion white** show the most **dramatic price spike** from late 2022 to early 2023, reaching levels well above 400 PHP/kg.  
  - This spike **does not directly align with typhoon clusters** but is more likely linked to **market and supply chain disruptions**, such as production shortages, storage issues, or import delays during that period.  
- After this surge, onion prices gradually stabilized but **remained more volatile** than other condiments, with smaller fluctuations in succeeding typhoon seasons.  
- **Garlic** prices stay relatively stable over time, showing only minor movements despite multiple typhoon events.  
- **Ginger** displays moderate fluctuations but lacks a strong or consistent pattern linked to typhoon clusters.  
- Unlike perishable vegetables or fruits, **condiment price volatility seems to be driven more by structural supply issues** (e.g., shortage, import policies) rather than **direct typhoon impacts**.  
- This suggests that **typhoons have a weaker immediate effect** on condiments compared to fresh produce, though **indirect effects** (e.g., logistics disruptions) may still contribute to some fluctuations.

In [121]:
dfs = []

for file in PRICE_FILES:
    path = os.path.join(CLEANED_DATA_DIR, file)
    if os.path.exists(path):
        df = pd.read_csv(path)
        dfs.append(df)
    else:
        print(f"⚠️ Missing file: {file}")

all_prices = pd.concat(dfs, ignore_index=True)

print(all_prices.head())

                                   Region Province         Commodity_Name  \
0  CAR - Cordillera Administrative Region     Abra                 CARROT   
1  CAR - Cordillera Administrative Region     Abra     TURNIP (SINGKAMAS)   
2  CAR - Cordillera Administrative Region     Abra      RADDISH (LABANOS)   
3  CAR - Cordillera Administrative Region     Abra                 POTATO   
4  CAR - Cordillera Administrative Region     Abra  SWEET POTATO (CAMOTE)   

   Unit  Year    Month  Retail_Price  
0  1 KG  2021  January        106.25  
1  1 KG  2021  January           NaN  
2  1 KG  2021  January           NaN  
3  1 KG  2021  January         93.75  
4  1 KG  2021  January         51.25  


# Price Spike

### Price Spike Detection Method

To measure price volatility, we identified **price spikes** using the Interquartile Range (IQR) method. This helps detect days when prices increased unusually compared to their normal levels.

First, we calculated:

$$IQR = Q3 - Q1$$

A price is considered a spike if it goes above this threshold:

$$\text{Spike Threshold} = \text{Median} + 1.5 \times IQR$$

If the price on day \(t\), \(p_t\), is greater than this threshold, we mark it as a spike day. We then compute:

$$\text{Spike Percentage} = \frac{\text{Number of Spike Days}}{\text{Total Days}} \times 100\%$$

This shows how often a commodity experiences sudden price increases, which can be linked to supply disruptions caused by typhoons.


In [148]:
stats = all_prices.groupby('Commodity_Name')['Retail_Price'].agg(
    median='median',
    q75=lambda x: x.quantile(0.75),
    q25=lambda x: x.quantile(0.25)
).reset_index()
stats['IQR'] = stats['q75'] - stats['q25']

df_featured = all_prices.merge(stats, on='Commodity_Name', how='left')
df_featured['Price_Spike'] = df_featured['Retail_Price'] > (df_featured['median'] + 1.5 * df_featured['IQR'])

spike_summary = df_featured.groupby('Commodity_Name')['Price_Spike'].agg(
    Spike_Percentage=lambda x: x.mean() * 100
).reset_index().sort_values('Spike_Percentage', ascending=False)

spike_summary.head(10)

,Commodity_Name,Spike_Percentage
28,ONION RED CREOLE (BERMUDA RED),6.339286
36,TOMATO,6.321429
21,"FRESH FRUIT, PINEAPPLE, PINYA, MEDIUM",6.125000
24,"GINGER, LOOSE",5.875000
7,"EGGPLANT, LONG, PURPLE",5.839286
1,"BITTER GOURD (AMPALAYA), FRUIT",5.000000
2,CABBAGE,4.625000
3,CARROT,4.464286
34,SQUASH,3.214286
4,CHAYOTE,2.892857


In [265]:
top10_spikes = spike_summary.head(10)

sort_order = alt.EncodingSortField(
    field='Spike_Percentage',
    op='max',
    order='descending'
)

bars = alt.Chart(top10_spikes).mark_bar(size=22).encode(
    x=alt.X('Spike_Percentage:Q', title='Spike Frequency (%)'),
    y=alt.Y('Commodity_Name:N', sort=sort_order, title=None),
    color=alt.Color(
        'Spike_Percentage:Q',
        scale=alt.Scale(scheme='blues'),
        legend=alt.Legend(title='Spike Frequency (%)')
    ),
    tooltip=[
        alt.Tooltip('Commodity_Name:N', title='Commodity'),
        alt.Tooltip('Spike_Percentage:Q', format='.2f', title='Spike %')
    ]
)

text = alt.Chart(top10_spikes).mark_text(
    align='left',
    baseline='middle',
    dx=4,
    fontSize=12,
    color='black'
).encode(
    x='Spike_Percentage:Q',
    y=alt.Y('Commodity_Name:N', sort=sort_order),
    text=alt.Text('Spike_Percentage:Q', format='.2f')
)

spike_chart = (bars + text).properties(
    title='Top 10 Commodities by Price Spike Frequency',
    width=650,
    height=380
).configure_title(
    anchor='middle',
    fontSize=18,
    fontWeight='bold'
).configure_axis(
    labelFontSize=12,
    titleFontSize=13
).configure_view(
    strokeWidth=0
).interactive(bind_y=False)

spike_chart.display()

alt.LayerChart(...)

### Analysis of Price Spike Frequency

Based on the computed results, **Red Onion (Bermuda Red)** has the highest spike percentage at **6.34%**, making it the commodity with the most frequent price spikes in the dataset. This is followed closely by **Tomato (6.32%)**, **Pineapple (6.13%)**, and **Ginger (5.88%)**, which also exhibit notable price volatility.

**Eggplant (5.84%)** and **Bitter Gourd/Ampalaya (5.00%)** also show recurring price spikes, suggesting that these commodities are vulnerable to supply fluctuations, seasonal impacts, and distribution challenges.

In the mid-range of spike frequency, **Carrot (4.46%)** and **Cabbage (4.63%)** demonstrate moderate pricing irregularities compared to the top-ranking commodities.

Meanwhile, **Squash (3.21%)** and **Chayote (2.89%)** have the lowest spike percentages among the top 10, indicating relatively stable pricing throughout the observed period.

Overall, spike percentages range from **2.89% to 6.34%**, confirming that all of the commodities in the top 10 experienced repeated price spikes, although with varying frequency. This highlights the sensitivity of key agricultural products—particularly vegetables—to changes in supply conditions, weather disturbances, and market dynamics.

In [244]:
region_total_spikes = (
    df_featured[df_featured['Price_Spike'] == True]
    .groupby('Region')
    .size()
    .reset_index(name='Total_Spikes')
    .sort_values('Total_Spikes', ascending=False)
)

top_region_spikes_with_count = region_total_spikes[['Region', 'Total_Spikes']]

top_region_spikes_with_count.head(5)

,Region,Total_Spikes
4,III - Central Luzon,716
7,MIMAROPA Region,539
5,IV-A - CALABARZON,472
9,V - Bicol Region,412
3,II - Cagayan Valley,310


### Regional Distribution of Price Spikes

The table shows the **top 5 regions with the highest number of price spikes**. **III - Central Luzon** leads with **716 spikes**, followed by **MIMAROPA Region (539)**, **IV-A - CALABARZON (472)**, **V - Bicol Region (412)**, and **II - Cagayan Valley (310)**. These regions account for a **large share of the total observed spikes**, indicating that they experience **more frequent market disruptions** compared to other regions. Monitoring these areas could help **identify patterns in price volatility and prioritize intervention efforts**.›

# Volatility

In [156]:
volatility = (
    all_prices.groupby('Commodity_Name')['Retail_Price']
    .std()
    .reset_index()
    .rename(columns={'Retail_Price': 'Price_Std'})
)

top_10_volatility = volatility.sort_values(by='Price_Std', ascending=False).head(10)
top_10_volatility

,Commodity_Name,Price_Std
28,ONION RED CREOLE (BERMUDA RED),70.821534
29,ONION WHITE (YELLOW GRANEX),70.323161
16,"FRESH FRUIT, LANZONES",55.173250
24,"GINGER, LOOSE",44.088391
12,"FRESH FRUIT, GRAPES",40.599228
18,"FRESH FRUIT, MANGO, KALABAW, RIPE, MEDIUM",39.564523
19,"FRESH FRUIT, MANGO, PIKO, RIPE, MEDIUM",39.025097
8,"FRESH FRUIT, AVOCADO",38.019752
3,CARROT,36.702862
32,POTATO,35.844211


In [205]:
final_bar_chart = (
    alt.Chart(top_10_volatility)
    .mark_bar(size=22) 
    .encode(
        y=alt.Y(
            'Commodity_Name:N',
            sort=alt.EncodingSortField(
                field='Price_Std', 
                op='mean', 
                order='descending'
            ),
            title=None
        ),
        x=alt.X(
            'Price_Std:Q',
            title='Price Volatility (Standard Deviation - PHP/KG)'
        ),
        color=alt.Color(
            'Price_Std:Q',
            scale=alt.Scale(
                scheme='blues'
            ),
            title='Std Dev'
        ),
        tooltip=[
            alt.Tooltip('Commodity_Name:N', title='Commodity'),
            alt.Tooltip('Price_Std:Q', format='.2f', title='Std Dev (PHP)')
        ]
    )
)

text_layer = (
    alt.Chart(top_10_volatility)
    .mark_text(
        align='left',
        baseline='middle',
        dx=4,
        fontSize=12,
        color='black'
    )
    .encode(
        y=alt.Y(
            'Commodity_Name:N',
            sort=alt.EncodingSortField(
                field='Price_Std', 
                order='descending'
            )
        ),
        x='Price_Std:Q',
        text=alt.Text('Price_Std:Q', format='.2f')
    )
)

final_chart = (
    (final_bar_chart + text_layer)
    .properties(
        title='Top 10 Most Volatile Commodities: Price Standard Deviation',
        width=650,
        height=380
    )
    .configure_title(
        anchor='middle',  
        fontSize=18,
        fontWeight='bold'
    )
    .configure_axis(
        labelFontSize=12,
        titleFontSize=13
    )
    .configure_view(
        strokeWidth=0
    )
    .interactive(bind_y=False)
)

final_chart.display()

alt.LayerChart(...)

### Analysis of Price Volatility (Standard Deviation)

Based on the standard deviation of retail prices, **Red Onion (Bermuda Red)** has the highest price volatility with a `Price_Std` of **70.82**, followed closely by **White Onion (Yellow Granex)** at **70.32**. These two types of onions show the largest fluctuations in price over time among all commodities analyzed.

Several fruits also appear prominently in the top 10 most volatile commodities: **Lanzones (55.17)**, **Grapes (40.60)**, **Mango – Kalabaw variety (39.56)**, **Mango – Piko variety (39.03)**, and **Avocado (38.02)**. These results highlight that fruit prices—particularly seasonal fruits—tend to vary widely during the observed period.

Among the vegetables, **Ginger (44.09)**, **Carrot (36.70)**, and **Potato (35.84)** also show notable price variability, though still lower compared to onions.

Overall, the commodities within the top 10 show substantial price variation, with standard deviation values ranging from **35.84 to 70.82**. This indicates that their prices did not remain stable and fluctuated frequently throughout the study period, reflecting sensitivity to supply conditions, seasonality, and market dynamics.

# Price Lag

In [169]:
if 'Date' not in df_featured.columns and 'Year' in df_featured.columns and 'Month' in df_featured.columns:
    df_featured['Date'] = pd.to_datetime(
        df_featured['Year'].astype(str) + ' ' + df_featured['Month'], 
        format='%Y %B',
        errors='coerce'
    )

df_typhoons.columns = df_typhoons.columns.str.strip()
df_typhoons['Typhoon_Date'] = pd.to_datetime(df_typhoons['Date_Entered_PAR'], errors='coerce')
df_typhoons['Typhoon_Name'] = df_typhoons['Typhoon Name']

df_featured = df_featured.sort_values(['Commodity_Name', 'Date'])
df_typhoons = df_typhoons.sort_values('Typhoon_Date')

lags = []

for _, typhoon in df_typhoons.iterrows():
    typhoon_date = typhoon['Typhoon_Date']
    typhoon_name = typhoon['Typhoon_Name']

    if pd.isna(typhoon_date):
        continue

    window_end = typhoon_date + pd.DateOffset(months=2)
    spikes_after = df_featured[
        (df_featured['Price_Spike'] == True) &
        (df_featured['Date'] >= typhoon_date) &
        (df_featured['Date'] <= window_end)
    ]

    if not spikes_after.empty:
        first_spikes = spikes_after.groupby('Commodity_Name')['Date'].min().reset_index()

        first_spikes['Lag_Months'] = (
            (first_spikes['Date'].dt.year - typhoon_date.year) * 12 +
            (first_spikes['Date'].dt.month - typhoon_date.month)
        )

        first_spikes['Typhoon_Name'] = typhoon_name
        lags.append(first_spikes)


In [171]:
df_lag = pd.concat(lags, ignore_index=True)
lag_summary = df_lag.groupby('Commodity_Name')['Lag_Months'].agg(['mean', 'median', 'count']).reset_index()
lag_summary = lag_summary.sort_values('mean')

lag_summary.head(10)

,Commodity_Name,mean,median,count
21,"GARLIC, LOCAL",0.900000,1.0,10
13,"FRESH FRUIT, GUYABANO",0.956522,1.0,23
19,"FRESH FRUIT, PINEAPPLE, PINYA, MEDIUM",0.969697,1.0,66
14,"FRESH FRUIT, LANZONES",1.000000,1.0,19
32,SQUASH,1.000000,1.0,61
24,MALUNGGAY,1.000000,1.0,34
7,"EGGPLANT, LONG, PURPLE",1.000000,1.0,59
9,"FRESH FRUIT, BANANA, LAKATAN, MEDIUM",1.025000,1.0,40
31,RADDISH (LABANOS),1.034483,1.0,58
3,CARROT,1.036364,1.0,55


### Price Lag Analysis

Based on the table, all listed commodities have a **mean price lag between 0.90 and 1.036**, with **median = 1.0** for all items. This indicates that **prices adjust very quickly, generally within 1 month after a typhoon-related disruption**. The **count of observations** ranges from 10 (Garlic) to 66 (Pineapple), showing that the dataset has **sufficient data for most commodities**. Commodities such as **Garlic, Guyabano (Guava), Pineapple, Lanzones, Squash, Malunggay, Eggplant, Lakatan Banana, Raddish, and Carrot** all exhibit **mean values close to 1**, demonstrating that **price response is consistently fast across the dataset**. Overall, the data shows **minimal variation in price lag among these commodities**, with all prices reacting quickly according to the observed mean and median values.

In [209]:
lag_summary.tail(5)

,Commodity_Name,mean,median,count
30,POTATO,1.277778,1.0,18
25,OKRA,1.279070,1.0,43
8,"FRESH FRUIT, AVOCADO",1.322581,1.0,31
23,KANGKONG,1.375000,1.0,8
35,TURNIP (SINGKAMAS),1.375000,1.0,16


### Price Lag Analysis

The table lists the **five commodities with the highest mean price lag** in the dataset. Their **mean price lags range from 1.2778 to 1.375**, while the **median for all items remains 1.0**, indicating that **prices generally adjust within 1–2 months after a typhoon-related disruption**. These commodities are **Potato, Okra, Fresh Fruit (Avocado), Kangkong, and Turnip (Singkamas)**, with observation counts ranging from 8 (Kangkong) to 43 (Okra). Compared to the rest of the dataset, these items exhibit **slightly slower price response**. Overall, the data shows that even the slowest-adjusting commodities **still react relatively quickly**, though there is **more variation in lag among these items** than among the fastest-adjusting commodities.

In [231]:
df_lag = lag_summary.rename(columns={'mean': 'Lag_Months'}).copy()

df_resilience_data = volatility.merge(
    spike_summary, 
    on='Commodity_Name', 
    how='inner'
)

df_resilience_data = df_resilience_data.merge(
    df_lag[['Commodity_Name', 'Lag_Months']], 
    on='Commodity_Name', 
    how='inner'
)

df_resilience_data.head()

,Commodity_Name,Price_Std,Spike_Percentage,Lag_Months
0,ALUGBATI,14.361489,0.553571,1.230769
1,"BITTER GOURD (AMPALAYA), FRUIT",25.775088,5.000000,1.076923
2,CABBAGE,30.869092,4.625000,1.084746
3,CARROT,36.702862,4.464286,1.036364
4,CHAYOTE,16.329441,2.892857,1.078125


In [233]:
base = alt.Chart(df_resilience_data)

bubble_layer = base.mark_circle().encode(
    x=alt.X('Lag_Months:Q', 
            title='Mean Price Lag (Months) after Typhoon', 
            scale=alt.Scale(domain=[0, 2]),
            axis=alt.Axis(grid=True)), 
    
    y=alt.Y('Price_Std:Q', 
            title='Price Volatility (Standard Deviation)', 
            scale=alt.Scale(domain=[30, 75]),
            axis=alt.Axis(grid=True)),
    
    size=alt.Size('Spike_Percentage:Q', title='Spike Frequency (%)'),
    
    color=alt.Color('Commodity_Name:N', legend=alt.Legend(columns=2)),
    
    tooltip=[
        'Commodity_Name:N',
        alt.Tooltip('Price_Std:Q', format='.2f', title='Volatility (Std Dev)'),
        alt.Tooltip('Spike_Percentage:Q', format='.2f', title='Spike Frequency (%)'),
        alt.Tooltip('Lag_Months:Q', format='.1f', title='Lag (Months)')
    ]
).properties(
    title='Commodity Resilience: Volatility vs. Price Lag Matrix'
)
 
text_labels = base.mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    fontSize=10,
    color='black'
).encode(
    x='Lag_Months:Q',
    y='Price_Std:Q',
    text='Commodity_Name:N',
    size=alt.value(0)
)

vertical_rule = alt.Chart(pd.DataFrame({'Lag_Months': [1.0]})).mark_rule(
    strokeDash=[5, 5], color='red', opacity=0.7
).encode(x='Lag_Months')

horizontal_rule = alt.Chart(pd.DataFrame({'Price_Std': [50]})).mark_rule(
    strokeDash=[5, 5], color='red', opacity=0.7
).encode(y='Price_Std')

final_bubble_chart = alt.layer(
    bubble_layer, 
    text_labels, 
    vertical_rule, 
    horizontal_rule
).interactive().resolve_scale(
    color='independent'
)

final_bubble_chart.display()

alt.LayerChart(...)

### Commodity Resilience Analysis

This analysis interprets **Mean Price Lag (X-axis)** as the **time (in months) from the typhoon event to the commodity's first major price spike**. **Volatility (Y-axis)** is the price stability after the spike.

### Least Resilient

* **ONION RED CREOLE** and **ONION WHITE** are major outliers. They combine **high volatility** (Price Std. Dev. $\ge 50$ to around 70) with a **slow spike response** (Lag $\ge 1.0$ month).
* **Implication:** The market takes a month or more to fully register the maximum price shock, and the resulting price movement remains **highly unstable** thereafter.
* **Most Turbulent:** The single highest-volatility item (Y $\approx 70$) has the **highest spike frequency**. Its spike is delayed slightly (Lag $\approx 1.0$ month) but results in the most extreme price movement.

### Slow Shock Transmission

* The **majority of commodities** (dense cluster in the Bottom-Right) are characterized by a **delayed price spike** (Lag $\ge 1.0$ month) but **low subsequent volatility** (Price Std. Dev. $< 50$).
* **Implication:** The supply shock is transmitted slowly through the supply chain, but once the spike occurs, the price stability is relatively good.

### Most Resilient (Fast Shock Absorption)

* Commodities like **CARROT** and **GINGER** are the best performers. They show a **fast price spike** (Lag $< 1.0$ month), but the resulting price movement is **stable** (low volatility).
* **Implication:** The market quickly registers the maximum price shock and then immediately settles into a stable, post-typhoon price environment.

In [222]:
df_spikes = df_featured[df_featured['Price_Spike'] == True]

regional_spike_counts = df_spikes.groupby(['Region', 'Commodity_Name']).size().reset_index(name='Commodity_Spike_Count')

region_totals = df_spikes.groupby('Region').size().reset_index(name='Total_Region_Spikes')

regional_spike_contribution = regional_spike_counts.merge(
    region_totals, 
    on='Region', 
    how='left'
)

regional_spike_contribution['Spike_Contribution_Pct'] = (
    regional_spike_contribution['Commodity_Spike_Count'] / regional_spike_contribution['Total_Region_Spikes'] * 100
)

high_risk_regions = ['I - Ilocos Region', 'Negros Island Region (NIR)', 'V - Bicol Region', 'MIMAROPA Region', 'IV-A - CALABARZON']

top_regional_spikes = regional_spike_contribution[
    regional_spike_contribution['Region'].isin(high_risk_regions)
].sort_values(
    ['Region', 'Spike_Contribution_Pct'], 
    ascending=[True, False]
).groupby('Region').head(3)

print("Top 3 Commodity Spike Contributors by High-Risk Region:")
top_regional_spikes

Top 3 Commodity Spike Contributors by High-Risk Region:


,Region,Commodity_Name,Commodity_Spike_Count,Total_Region_Spikes,Spike_Contribution_Pct
30,I - Ilocos Region,"FRESH FRUIT, PINEAPPLE, PINYA, MEDIUM",48,103,46.601942
35,I - Ilocos Region,TOMATO,12,103,11.650485
33,I - Ilocos Region,ONION RED CREOLE (BERMUDA RED),10,103,9.708738
88,IV-A - CALABARZON,"FRESH FRUIT, PINEAPPLE, PINYA, MEDIUM",89,472,18.855932
94,IV-A - CALABARZON,SQUASH,60,472,12.711864
85,IV-A - CALABARZON,"EGGPLANT, LONG, PURPLE",44,472,9.322034
106,MIMAROPA Region,CABBAGE,78,539,14.471243
107,MIMAROPA Region,CARROT,61,539,11.317254
109,MIMAROPA Region,COMMON GOURD (UPO),55,539,10.204082
130,Negros Island Region (NIR),"FRESH FRUIT, BANANA, SABA, MEDIUM",39,136,28.676471


### Overall Insight

- **High-risk commodities identified:** Red onion, tomato, ginger, and eggplant show **frequent price spikes, high volatility, and rapid price reactions**, making them highly vulnerable to typhoon disruptions.
- **Consumer impact:** These essential vegetables are part of daily Filipino meals, so their unstable prices **directly affect household food affordability**, especially after typhoons.
- **Regional exposure:** Regions frequently hit by typhoons (Ilocos, Bicol, NIR, MIMAROPA, CALABARZON) show **higher counts of price spikes**, confirming a strong climate–price connection.
- **Price adjustment lag:** Most commodities **respond within one month** after a supply shock, showing **fast price transmission** from farm damage to market prices.
- **Policy relevance:** Results highlight the need to **strengthen storage facilities, diversify supply chains, and improve price monitoring** to reduce post-typhoon market instability.

**Conclusion:** Typhoons significantly amplify agricultural price instability in the Philippines, affecting both producers and consumers. Understanding commodity behavior under climate shocks is crucial for food security planning.
